<img src="gas.png" alt="Gas" align="left" style="width: 60px;"/> 
<img src="uem.png" alt="Drawing" align="right" style="width: 300px;"/> 

--- 
<center> <font  size="10" color="#de3831">
    <b> EXAMEN FINAL DE PEL </b>
</font> </center>

<center> <font  size="3" color="black">
 1. Determinar cuántas estaciones de servicios hay en el código postal de la explosión <br />
    </font> </center>

<center>  <font  size="5" color="black">  Maria Sol Torres Lopez </center> 

---
 <font size="6" color="black">
1. CREAR Y CONFIGURAR EL AMBIENTE
</font> 

#### Configurar las librerias externas especificando los paths donde estan y cargandolas

In [1]:
#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libcurl")

#pragma cling add_library_path("/Users/sol/opt/miniconda3/envs/cling/lib")
#pragma cling add_include_path("/Users/sol/opt/miniconda3/envs/cling/include")
#pragma cling load("libjsoncpp")

#### Incluir las librerias necesarias

In [2]:
#include <iostream>
#include <fstream>
#include <string>
#include <array>
#include <algorithm>

//Curl
#include <curl/curl.h>

//Json
#include <json/json.h>
#include <json/reader.h>
#include <json/writer.h>
#include <json/value.h>

//Mapas
#include "xleaflet/xmap.hpp"
#include "xleaflet/xtile_layer.hpp"
#include "xleaflet/xmarker.hpp"
#include "xleaflet/xmarker_cluster.hpp"
#include "xleaflet/xicon.hpp"
#include "xleaflet/xbasemaps.hpp"
#include "xleaflet/xpopup.hpp"
#include "xwidgets/xhtml.hpp"

using namespace std;


<font size="6" color="black">
2. ESTRUCTURA PARA ALMACENAR DATOS
</font> 

#### Creacion de la estructura guardar los datos de las estaciones de servicio

In [3]:
struct estacion {
    string provincia;
    string municipio;
    string localidad;
    string codigoPostal;
    string direccion;
    string margen;
    double longitud;
    double latitud;
    double precio;
    string rotulo;

    estacion *next;
};

<font size="6" color="black">
3. FUNCIONES
</font> 

* Funcion para enlistar una estacion a una lista de estaciones

In [4]:
void push(estacion*& lista, estacion* gasolinera);

In [5]:
void push(estacion*& lista, estacion* gasolinera) {
    //Se crea un nuevo nodo para agregar a la lista
    estacion* aux = new estacion();
    
    //Se guardan los atributos del puntero "gasolinera" en el nuevo nodo 
    aux->provincia = gasolinera->provincia;
    aux->municipio = gasolinera->municipio;        
    aux->localidad = gasolinera->localidad;
    aux->codigoPostal= gasolinera->codigoPostal;
    aux->direccion= gasolinera->direccion;
    aux->margen= gasolinera->margen;
    aux->longitud = gasolinera->longitud;
    aux->latitud = gasolinera->latitud;
    aux->precio = gasolinera->precio;
    aux->rotulo= gasolinera->rotulo;
    
    //Se enlista el nodo conectando con el ultimo de la lista y actualizando el final de la lista 
    if (lista==NULL){
        lista = aux;
        aux->next = NULL;
    }else{
        aux->next = lista;
        lista = aux;
    }
}

* Manejar la informacion de respuesta del GET REQUEST

In [6]:
size_t WriteCallback(char* contenido, size_t size, size_t num, string* out);

In [7]:
static size_t WriteCallback(char* contenido, size_t size, size_t num, string* out) {
    //Lo que se este recibiendo, se va agregando al resultado final de la respuesta
    out -> append(contenido, size * num);
        return size * num;
}

* Transformar a double los datos numericos

In [8]:
double toDouble(string s); 

In [9]:
double toDouble(string s) {
    //Reemplaza las comas por puntos de un numero en string
    replace(s.begin(), s.end(), ',', '.');
     //y luego lo transforma a double y lo devuelve como resultado
        return atof(s.c_str());
}

* Transformar del string que se obtiene de la lectura de la API a un objeto JSON

In [10]:
Json::Value ParseJson(string content);

In [11]:
static Json::Value ParseJson(string content) {
    //Objetos para leer los caracteres del string y transformar a Json 
    Json::CharReaderBuilder builder;
    Json::CharReader* reader = builder.newCharReader();
    
    //Variable donde se va a almacenar los datos transformados a JSON
    Json::Value jsonData;
    //Variable para guardar los psoibles errores que puedan ocurrir al parsear
    string errors;
    
    //Se hace la lectura y parseo del contenido en string con la funcion parse, la cual devuelve un boolean segun exito o fracaso de la operacion
    bool exitoParseando = reader->parse(content.c_str(), content.c_str() + content.size(), &jsonData, &errors);
    
    //Al no necesitar al reader, se lo elimina
    delete reader;
    
    //Si el resultado del parseo no es exitoso, se imprime un mensaje informandolo y mostrando los datos en raw
    if (!exitoParseando) {
        cout << "Error transfromando la informacion HTTP a JSON" << std::endl;
        cout << "Los datos del HTTP eran:\n" << content << std::endl;
        return 0;
    }
    //En caso de que sea exitoso, se extrae la fecha del JSON y se muestra
    else {
        const string dateString(jsonData["Fecha"].asString());

       std::cout << "Datos actualizados del: " << jsonData["Fecha"] << std::endl;
        std::cout << std::endl;
    }
    return jsonData;
    
}

* Hace el request de la informacion y guarda las estaciones en una lista

In [12]:
void getGasolineras(int tipo, estacion *&lista);

In [13]:
void getGasolineras(int tipo, estacion *&lista) {
    //Aqui se guardara el ID del producto segun el listado de la pagina web
    string IDProducto;
    
    //Segun la opcion que se haya ingresado en el menu, se asigna el ID
    if(tipo == 1) {
        IDProducto = "1";
    }
    if(tipo == 2){
        IDProducto = "23";
    }
    if(tipo == 3){
        IDProducto = "20";
    }
    if (tipo == 4){
       IDProducto = "3";
    }
    if (tipo == 5){
        IDProducto = "21";
    }
    if (tipo == 6){
        IDProducto = "4";
    }
    if (tipo == 7){
        IDProducto = "5";    
    }
    if (tipo == 8){
        IDProducto = "6";
    }
    
    //Si la opcion es mayor a 9, no es valido
    if (tipo < 9) {
    
    //Se define la URL para pedir los datos
    const std::string url("https://sedeaplicaciones.minetur.gob.es/ServiciosRESTCarburantes/PreciosCarburantes/EstacionesTerrestres/FiltroProducto/" + IDProducto);
    CURL* curl = curl_easy_init();
    string readBuffer;
    long httpCode(0);
    
    if (curl) {
        // Configurar como request GET
        curl_easy_setopt(curl, CURLOPT_CUSTOMREQUEST, "GET");
        // Configurar el URL.
        curl_easy_setopt(curl, CURLOPT_URL, url.c_str());
        // Para que no intente IPv6, porque aumentaria el tiempo de resolucion de DNS.
        curl_easy_setopt(curl, CURLOPT_IPRESOLVE, CURL_IPRESOLVE_V4);
        //Para que solo espere maximo 10 segundos la respuesta
        curl_easy_setopt(curl, CURLOPT_TIMEOUT, 10);
        
        // Enlazar la funcion que va a manejar la informacion de respuesta
        curl_easy_setopt(curl, CURLOPT_WRITEFUNCTION, WriteCallback);
        
        // Enlazar un contenedor para la informacion (will be passed as the last parameter to the
        // callback handling function).  
        curl_easy_setopt(curl, CURLOPT_WRITEDATA, &readBuffer);
        
        // Ejecutar el comando HTTP GET, y guardar el codigo de respuesta HTTP en httpCode
        curl_easy_perform(curl);
        curl_easy_getinfo(curl, CURLINFO_RESPONSE_CODE, &httpCode);
    }
    //Limpiar luego de terminar la solicitud
   curl_easy_cleanup(curl);
    
    //Si el codigo HTTP recuperado es 200, significa que la solicitud fue exitosa
    if (httpCode == 200)
    {
        //std::cout << "\nRespuesta exitosa de " << url << std::endl;
        // La respuesta correcta - terminado con lo de curl.  
        // Intentar parsear los resultados e imprimirlos
        Json::StreamWriterBuilder builder;
        
        //Se parsea la informacion recuperada de la web en String a JSON
        Json::Value jsonData = ParseJson(readBuffer);
        
        //Se seleccionan los valores bajo la key ListaEESSPrecio del JSON, que es donde esta la lista de estaciones
        const Json::Value& listaprecio = jsonData["ListaEESSPrecio"];
    
        //Se recorre la lista de estaciones con sus precios
        int i=0;
        while(i< listaprecio.size()) {
            //Se guarda cada estacion del JSON en un puntero nuevo
            estacion *gasolinera= new estacion();
    
            //Antes de guardar el municipio, hay que pasarlo a mayuscula para que se pueda hacer las busquedas en mayuscula
            string municipio = listaprecio[i]["Municipio"].asString();
            for_each(municipio.begin(), municipio.end(), [](char & c){
                c = ::toupper(c);
            });
            //Se guardan los valores dentro del puntero
            gasolinera->provincia = listaprecio[i]["Provincia"].asString();
            gasolinera->municipio = municipio;
            gasolinera->localidad = listaprecio[i]["Localidad"].asString();
            gasolinera->codigoPostal= listaprecio[i]["C.P."].asString();
            gasolinera->direccion= listaprecio[i]["Dirección"].asString();
            gasolinera->margen= listaprecio[i]["Margen"].asString();
            
            //Para el caso de los double, llamo a la funcion to Double para pasar de un string separado por coma, a double decimal por punto
            gasolinera->longitud = toDouble(listaprecio[i]["Longitud (WGS84)"].asString());
            gasolinera->latitud = toDouble(listaprecio[i]["Latitud"].asString());
            gasolinera->precio = toDouble(listaprecio[i]["PrecioProducto"].asString());
            gasolinera->rotulo= listaprecio[i]["Rótulo"].asString();
            
            //Se hace el enlistado
            if (lista==NULL){
                lista = gasolinera;
                gasolinera->next = NULL;
            }else{
                gasolinera->next = lista;
                lista = gasolinera;
            }
            i++;
        }
    }
    else
    {
        std::cout << "No se pudieron recuperar los datos de " << url << " - saliendo..." << std::endl;
        
    }
    }
}

* Busqueda de las estaciones de servicio (gasolineras) mas baratas

In [14]:
void gasolinera(estacion *lista, string provincia, string municipio, string localidad, estacion *&resultados);

In [15]:
void gasolinera(estacion *list, string provincia, string municipio, string localidad, estacion *&resultados, int cantidad){
    double precio=0.0;
    estacion *comprobar = list;
    //Puntero auxiliar para hacer el recorrido de la lista en la busqueda de los 5 menores
    estacion *header1 = list;
    
    //Se crea un nodo de referencia que este vacio en sus campos y con el precio muy alto, para usarlo como referencia en la busqueda de menores
    estacion *menor = new estacion();
    menor->provincia = "Sin provincia";
    menor->municipio = "Sin municipio";        
    menor->localidad = "Sin localidad";
    menor->codigoPostal= "Sin C.P.";
    menor->direccion= "Sin direccion";
    menor->margen= "NaN";
    menor->longitud = 0.0;
    menor->latitud = 0.0;
    menor->precio = 99999;
    menor->rotulo= "NaN";
    
    //Asignar todos los pequenos a sin datos
    estacion *peque1 = menor;
    estacion *peque2= menor;
    estacion *peque3= menor;
    estacion *peque4= menor;
    estacion *peque5= menor;
    estacion *peque6= menor;
    estacion *peque7= menor;
    estacion *peque8= menor;
    estacion *peque9= menor;
    estacion *peque10= menor;
    
    //Contadores para encontrar las coincidencias de la localizacion de la busqueda
    int contprovincia = 0;
    int contmunicipio = 0;
    int contlocalidad = 0;

    //comprobamos que la ubicacion ingresada existe en los datos recorriendo los datos
    do{
       //Si la provincia de la estacion* comprobar coincide con la provincia que se busca, se cuenta la coincidencia
        if(comprobar->provincia == provincia){
            contprovincia++;
        }
        //Si el municipio de la estacion* comprobar coincide con el municipio que se busca, se cuenta la coincidencia
        if(comprobar->municipio == municipio){
            contmunicipio++;
        }
        //Si la localidad de la estacion* comprobar coincide con la localidad que se busca, se cuenta la coincidencia
        if(comprobar->localidad == localidad){
            contlocalidad++;
        }
        comprobar = comprobar->next;
    }while(comprobar != NULL);
    
    //Si se ha encontrado alguna coincidencia en los tres campos, significa que hay al menos una gasolinera y se comienza la busqueda
    if(contprovincia != 0 && contmunicipio != 0 && contlocalidad != 0) {
        cout<<"Buscando gasolineras localizacion indicada..."<<endl;
    }
    //Si no se ha encontrado coincidencias en cualquier campo, significa que nohay gasolinera en esa provincia/municipio/localidad
    else if(contprovincia == 0 || contmunicipio == 0 || contlocalidad == 0){
        cout<<"Estos datos no se encuentran, asegurese de que estan correctamente escritos (Mayusculas)"<<endl;
        return;
    }
    
    //Para el mas barato
    while(header1!=NULL){
        //Si esta en la localizacion que se busca y el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque1), entra
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque1->precio)) {
            //Al ser el header1 menor que peque1, se actualiza peque1 como el nuevo mas pequenio
            peque1= header1;
        }
        header1 = header1->next;
    }
   
    //Si continua la ejecucion, se enlista peque1 en la lista de resultados y se imprime por consola
    push(resultados, peque1);
    cout<<"Calle 1: "<< peque1->direccion<<"   Precio: "<<peque1->precio <<endl;
    
    //Si en el conteo de localidades, se queda en 1, significa que solo hay una coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 1){
        cout<<"En "<<localidad<<" solo existe una gasolinera."<<endl;
        return;
    }
    
    //Si en el parametro int cantidad es 1, significa que solo quiero la mas barata
    if (cantidad ==1) {
        return;
    }

    //para el segundo mas barato
    //Se reinicializa el header para empezar otra vez el recorrido en el inicio de la lista
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque2->precio) && header1!= peque1) {
            //Al ser el header1 menor que peque2, se guarda header1 en peque2 como el nuevo segundo mas pequenio
            peque2= header1;
        }
        header1 = header1->next;
    }
    
    //Se enlista peque2 en la lista de resultados y se imprime por consola
    push(resultados, peque2);
    cout<<"Calle 2: "<< peque2->direccion<<"   Precio: "<<peque2->precio<<endl;
    
    //Si en el conteo de localidades se queda en 2, significa que solo hay dos coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 2){
        cout<<"En "<<localidad<<" solo existen dos gasolineras."<<endl;
        return;
    }

    //para el tercer mas barato
    //Se reinicializa el header para empezar otra vez el recorrido en el inicio de la lista
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer y segundo menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque3->precio) && header1 != peque1 && header1 != peque2) {
            //Al ser el header1 menor que peque3,  se guarda header1 en peque3 como el nuevo tercer mas pequenio
            peque3= header1;
        }
        header1 = header1->next;
    }
    
    //Se enlista peque3 en la lista de resultados y se imprime por consola
    push(resultados, peque3);
    cout<<"Calle 3: "<< peque3->direccion<<"   Precio: "<<peque3->precio<<endl;
    
    //Si en el conteo de localidades se queda en 3, significa que solo hay tres coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 3){
        cout<<"En "<<localidad<<" solo existen tres gasolineras."<<endl;
        return;
    }

    //para el cuarto mas barato
    //Se reinicializa el header para empezar otra vez el recorrido en el inicio de la lista
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo tercer menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque4->precio) && header1 != peque1 && header1 != peque2 && header1 != peque3) {
            //Al ser el header1 menor que peque4, se guarda header1 en peque4 como el nuevo tercer mas pequenio
            peque4= header1;
        }
        header1 = header1->next;
    }
    
    //Se enlista peque4 en la lista de resultados y se imprime por consola
    push(resultados, peque4);
    cout<<"Calle 4: "<< peque4->direccion<<"   Precio: "<< peque4->precio<<endl;
    
    //Si en el conteo de localidades se queda en 4, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 4){
        cout<<"En "<<localidad<<" solo existen cuatro gasolineras."<<endl;
        return;
    }

    //calculamos el pequeño 5
    //Se reinicializa el header para empezar otra vez el recorrido en el inicio de la lista
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque5->precio) && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque5= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque5);
    cout<<"Calle 5: "<< peque5->direccion<<"   Precio: "<<peque5->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 5){
        cout<<"En "<<localidad<<" solo existen cinco gasolineras."<<endl;
        return;
    }
    
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque6->precio) 
           && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4 && header1 != peque5) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque6= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque6);
    cout<<"Calle 6: "<< peque6->direccion<<"   Precio: "<<peque6->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 6){
        cout<<"En "<<localidad<<" solo existen seis gasolineras."<<endl;
        return;
    }
    
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque7->precio) 
           && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4 && header1 != peque5 && header1 != peque6) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque7= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque7);
    cout<<"Calle 7: "<< peque7->direccion<<"   Precio: "<<peque7->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 7){
        cout<<"En "<<localidad<<" solo existen siete gasolineras."<<endl;
        return;
    }
    
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque8->precio) 
           && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4 && header1 != peque5 && header1 != peque6 && header1 != peque7) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque8= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque8);
    cout<<"Calle 8: "<< peque8->direccion<<"   Precio: "<<peque8->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 8){
        cout<<"En "<<localidad<<" solo existen ocho gasolineras."<<endl;
        return;
    }
    
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque9->precio) 
           && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4 && header1 != peque5 && header1 != peque6 && header1 != peque7 && header1 != peque8) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque9= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque9);
    cout<<"Calle 9: "<< peque9->direccion<<"   Precio: "<<peque9->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 9){
        cout<<"En "<<localidad<<" solo existen nueve gasolineras."<<endl;
        return;
    }
    
    header1 = list;
    while(header1!=NULL){
        //Si está en la localizacion que se busca, el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque2), 
        // y ademas es distinto al primer, segundo, tercer y cuarto menor que encontramos, cumple
        if((header1->provincia)==provincia && (header1->municipio)==municipio && (header1->localidad)==localidad && (header1->precio)<= (peque10->precio) 
           && header1 != peque1 && header1 != peque2 && header1 != peque3 && header1 != peque4 && header1 != peque5 && header1 != peque6 && header1 != peque7 && header1 != peque8 && header1 != peque9) {
            //Al ser el header1 menor que peque5, se guarda header1 en peque5 como el nuevo tercer mas pequenio
            peque10= header1;
        }
        header1 = header1->next;
    }

    //Se enlista peque5 en la lista de resultados y se imprime por consola
    push(resultados, peque10);
    cout<<"Calle 10: "<< peque10->direccion<<"   Precio: "<<peque10->precio<<endl;
    
    //Si en el conteo de localidades se queda en 5, significa que solo hay cuatro coincidencia en la ubicacion y se deja de buscar 
    if(contlocalidad == 10){
        cout<<"En "<<localidad<<" solo existen nueve gasolineras."<<endl;
        return;
    }
    
}

* Cantidad de estaciones en el codigo postal de la explosion

In [16]:
void conteoEstaciones(estacion *pila);

In [17]:
void conteoEstaciones(estacion *pila) {
    
    //Puntero auxiliar para hacer el recorrido de la lista en la busqueda de los 5 menores
    estacion *aux = pila;
    int conteo=0;
    
    //Para el conteo
    while(aux!=NULL){
        //Si esta en MADRID y el codigo postal del nodo actual es igual al de la explosion, entra
        if((aux->municipio)=="MADRID" && (aux->localidad)=="MADRID" && (aux->codigoPostal)== "28005") {
            conteo++;
        }
        aux = aux->next;
    }
    
    cout << "Hay " << conteo << " estaciones en el mismo codigo postal de la explosion" <<endl;
}

* Cantidad de estaciones low cost en Barcelona

In [18]:
void conteoEstacionesLowCost();

In [19]:
void conteoEstacionesLowCost() {
    //Obtener gasolineras de Gasoleo
    estacion *gasoleoA = NULL;
    getGasolineras(6, gasoleoA);
    
    //Puntero auxiliar para hacer el recorrido de la lista en la busqueda de los 5 menores
    estacion *aux = gasoleoA;
    int conteo=0;
    
    //Para el conteo
    while(aux!=NULL){
        //Si esta en BARCELONA y el precio del gasoleo es menos de 1 euro, entra
        if((aux->municipio)=="BARCELONA" && (aux->precio)< 1) {
            conteo++;
        }
        aux = aux->next;
    }
    
    cout << "Hay " << conteo << " estaciones con el Gasoleo A a menos de 1 euro en BARCELONA" <<endl;
}

* Estacion mas barata de gasoleo A

In [20]:
void BarataGasoleoA();

In [21]:
void BarataGasoleoA() {   
    //Obtener gasolineras de Gasoleo
    estacion *gasoleoA = NULL;
    getGasolineras(6, gasoleoA);
    
    //Se crea un nodo de referencia que este vacio en sus campos y con el precio muy alto, para usarlo como referencia en la busqueda de menores
    estacion *menor = new estacion();
    menor->provincia = "Sin provincia";
    menor->municipio = "Sin municipio";        
    menor->localidad = "Sin localidad";
    menor->codigoPostal= "Sin C.P.";
    menor->direccion= "Sin direccion";
    menor->margen= "NaN";
    menor->longitud = 0.0;
    menor->latitud = 0.0;
    menor->precio = 99999;
    menor->rotulo= "NaN";
    
    //Asignar todos los pequenos a sin datos
    estacion *peque1 = menor;
    
    //Puntero auxiliar para hacer el recorrido de la lista en la busqueda de los 5 menores
    estacion *aux = gasoleoA;
    
    //Para el mas barato
    while(aux!=NULL){
        //Si esta en la localizacion que se busca y el precio del nodo actual es menor que el referente mas pequenio que tenemos (peque1), entra
        if((aux->precio)<= (peque1->precio)) {
            //Al ser el header1 menor que peque1, se actualiza peque1 como el nuevo mas pequenio
            peque1= aux;
        }
        aux = aux->next;
    }
    cout<<"Provinvia: " << peque1->provincia << " Municipio: " << peque1-> municipio << "Calle: "<< peque1->direccion<<"   Precio: "<<peque1->precio <<endl;
    
}

* Pinta los los marcadores de los resultados

In [22]:
void pintarMapa(estacion* resultados);

<font size="6" color="black">
4. PROGRAMA
</font>

## Mapa

#### Se crea y muestra el mapa de españa

In [23]:
//Crear el mapa
xlf::map mymap;
mymap.zoom = 6;
//Coordenadas del centro de espana
mymap.center = std::array<double, 2>({40.4166400, -3.7032700});
mymap.display();

A Jupyter widget

* Definicion de la funcion pintarMapa

In [24]:
//Una vez creado el mapa, ya puedo definir esta funcion
void pintarMapa(estacion* resultados) {
    estacion* aux = resultados;
    estacion* coordinadas[10];
    int i = 0;

    //Crear array de los resultados para poder acceder a ellos facilmente y crear los marcadores
    while (aux!=NULL && i<10) {
        coordinadas[i]=aux;
        i++;
        aux = aux->next;                
        
    }
        
    //Crear los 5 marcadores para los 5 mas baratos
    //Siempre va a haber al menos 1 resultado, ya que es la condicion para llamar a esta funcion
   
    auto marker1 = xlf::marker::initialize()
        .location({coordinadas[0]->latitud, coordinadas[0]->longitud})
        .title(coordinadas[0]->direccion)
        .finalize();
    
    //Compruebo el tamanio caso por caso 
    //Si hay solo 1 resultado, solo agregar el primer marcador
    if (i==1) {
         auto marker_cluster = xlf::marker_cluster::initialize() 
            .markers({marker1})
            .finalize();
        //Agregar una capa de los marcadores agrupados al mapa para mostrar
        mymap.add_layer(marker_cluster);
    }
        
        
        
    //Si hay al menos 2 resultados, crear un segundo marcador
    if (i>1) {
        auto marker2 = xlf::marker::initialize()
            .location({coordinadas[1]->latitud, coordinadas[1]->longitud})
            .title(coordinadas[1]->direccion)
            .finalize();

        //Si hay solo 2 resultados, agregar solo el primer y segundo marcador 
         if (i==2) {
            auto marker_cluster = xlf::marker_cluster::initialize() 
                .markers({marker1, marker2})
                .finalize();
             
            //Agregar una capa de los marcadores agrupados al mapa para mostrar
                mymap.add_layer(marker_cluster);
         }
                
            //Si hay al menos 3 resultados, crear un tercer marcador
            if (i>2) {
                auto marker3 = xlf::marker::initialize()
                    .location({coordinadas[2]->latitud, coordinadas[2]->longitud})
                    .title(coordinadas[2]->direccion)
                    .finalize();

                //Si hay solo 3 resultados, agregar solo el primer, segundo y tercer marcador  
                if (i==3){
                    auto marker_cluster = xlf::marker_cluster::initialize() 
                        .markers({marker1, marker2, marker3})
                        .finalize();
                    
                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                    mymap.add_layer(marker_cluster);

                }
                    //Si hay al menos 4 resultados, crear un cuarto marcador
                    if (i>3) {
                        auto marker4 = xlf::marker::initialize()
                            .location({coordinadas[3]->latitud, coordinadas[3]->longitud})
                            .title(coordinadas[3]->direccion)
                            .finalize();

                        //Si hay solo 4 resultados, agregar solo el primer, segundo, tercer y cuarto marcador 
                        if (i==4) {
                            auto marker_cluster = xlf::marker_cluster::initialize() 
                                .markers({marker1, marker2, marker3, marker4})
                                .finalize();

                            //Agregar una capa de los marcadores agrupados al mapa para mostrar
                            mymap.add_layer(marker_cluster);
                        }
                        
                            //Si hay 5 resultados, crear el quinto marcador
                            if (i>4) {
                                auto marker5 = xlf::marker::initialize()
                                    .location({coordinadas[4]->latitud, coordinadas[4]->longitud})
                                    .title(coordinadas[4]->direccion)
                                    .finalize();

                                //Si hay 5 resultados, agregar los 5 marcadores
                                if (i==5) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                                
                                //Si hay 6 resultados, crear el sexto marcador
                            if (i>5) {
                                auto marker6 = xlf::marker::initialize()
                                    .location({coordinadas[5]->latitud, coordinadas[5]->longitud})
                                    .title(coordinadas[5]->direccion)
                                    .finalize();

                                //Si hay 6 resultados, agregar los 6 marcadores
                                if (i==6) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5, marker6})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                                
                                //Si hay 7 resultados, crear el septimo marcador
                                if (i>6) {
                                auto marker7 = xlf::marker::initialize()
                                    .location({coordinadas[6]->latitud, coordinadas[6]->longitud})
                                    .title(coordinadas[6]->direccion)
                                    .finalize();

                                //Si hay 7 resultados, agregar los 7 marcadores
                                if (i==7) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5, marker6, marker7})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                                
                                    //Si hay 8 resultados, crear el octavo marcador
                                if (i>7) {
                                auto marker8 = xlf::marker::initialize()
                                    .location({coordinadas[7]->latitud, coordinadas[7]->longitud})
                                    .title(coordinadas[7]->direccion)
                                    .finalize();

                                //Si hay 8 resultados, agregar los 8 marcadores
                                if (i==8) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5, marker6, marker7, marker8})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                                    
                                    //Si hay 9 resultados, crear el noveno marcador
                                if (i>8) {
                                auto marker9 = xlf::marker::initialize()
                                    .location({coordinadas[8]->latitud, coordinadas[8]->longitud})
                                    .title(coordinadas[8]->direccion)
                                    .finalize();

                                //Si hay 9 resultados, agregar los 8 marcadores
                                if (i==9) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5, marker6, marker7, marker8, marker9})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                                    //Si hay 10 resultados, crear el decimo marcador
                                if (i>9) {
                                auto marker10 = xlf::marker::initialize()
                                    .location({coordinadas[9]->latitud, coordinadas[9]->longitud})
                                    .title(coordinadas[9]->direccion)
                                    .finalize();

                                //Si hay 9 resultados, agregar los 8 marcadores
                                if (i==10) {
                                    auto marker_cluster = xlf::marker_cluster::initialize() 
                                        .markers({marker1, marker2, marker3, marker4, marker5, marker6, marker7, marker8, marker9, marker10})
                                        .finalize();
                                    //Agregar una capa de los marcadores agrupados al mapa para mostrar
                                    mymap.add_layer(marker_cluster);
                                }
                               }
                            }
                        }
                    }     
                 } 
              }
          }
      }
    }
        //Ajustar la vista del mapa cambiando su centro al de los resultados y ampliando el zoom
        mymap.center = std::array<double, 2>({resultados->latitud, resultados->longitud});
        mymap.zoom = 12;
            
    }


## Ejecucion del programa

In [ ]:
    
    string provinciaPedida, municipioPedido, localidadPedida;
    int tipoGasolina, repetir;

    //menu
    cout << "-------------------------------------------------" << endl;
    cout << "      Bienvenido al buscador de gasolineras" << endl;
    cout << "-------------------------------------------------" << endl;
    do{
        //header vacio para crear la lista de estaciones mas tarde segun la gasolina que se pida
        estacion *header = NULL;
        //Crear nueva lista de resultados vacia
        estacion *resultados= NULL;
        
        cout<<"Que tipo gasolina necesita?"<<endl;
        cout<<" Tipos gasolina:  "<<endl;
        cout<<" 1/ Gasolina 95 E5"<<endl;
        cout<<" 2/ Gasolina 95 E10"<<endl;
        cout<<" 3/ Gasolina 95 E5 Premium"<<endl;
        cout<<" 4/ Gasolina 98 E5"<<endl;
        cout<<" 5/ Gasolina 98 E10"<<endl;
        cout<<" 6/ Gasoleo A"<<endl;
        cout<<" 7/ Gasoleo Premium"<<endl;
        cout<<" 8/ Gasoleo B"<<endl;
        cin>>tipoGasolina;
        
        //Descargar la informacion de las gasolineras
        getGasolineras(tipoGasolina, header);
        
        //4. Cantidad de estaciones en el codigo postal
        cout << "CANTIDAD ESTACIONES EN EXPLOSION" <<endl;
        conteoEstaciones(header);
        cout << "\n" <<endl;
        
        //6. Estacion de servicio mas barata de Madrid
        cout << "ESTACION MAS BARATA DE MADRID" <<endl;
        gasolinera(header,"MADRID","MADRID","MADRID", resultados, 1);
        cout << "\n" <<endl;
        
        //5. 10 gasolineras mas baratas en Valdemoro
        cout << "10 GASOLINERAS MAS BARATAS EN VALDEMORO" <<endl;
        gasolinera(header,"MADRID","VALDEMORO","VALDEMORO", resultados, 10);
        cout << "\n" <<endl;
        
        //7. Cantidad de estaciones low cost en Barcelona
        cout << "ESTACIONES LOWCOST EN BARCELONA" <<endl;
        conteoEstacionesLowCost();
        cout << "\n" <<endl;
        
        //8. DONDE ES MAS BARATO EL GASOLEO A
        cout << "ESTACION MAS BARATA DE GASOLEO A" <<endl;
        BarataGasoleoA();
        cout << "\n" <<endl;
        
        //9. RESULTADOS EN EL MAPA
        //Si los resultados no estan vacios, se mostrara el mapa
        if (resultados!=NULL) {
            pintarMapa(resultados); 
            cout << "Los resultados de la busqueda pueden visualizarse en el mapa de la parte superior" <<endl;
        }
        
        cout<<"\nQuiere buscar otro tipo de gasolina? (si=1 / no=0)"<<endl;
        cin>>repetir;
    }while(repetir == 1);
    if(repetir == 0){
        cout<<"Hasta la proxima"<<endl;
    }


-------------------------------------------------
      Bienvenido al buscador de gasolineras
-------------------------------------------------
Que tipo gasolina necesita?
 Tipos gasolina:  
 1/ Gasolina 95 E5
 2/ Gasolina 95 E10
 3/ Gasolina 95 E5 Premium
 4/ Gasolina 98 E5
 5/ Gasolina 98 E10
 6/ Gasoleo A
 7/ Gasoleo Premium
 8/ Gasoleo B


<center> <font size="6" color="black">
    <b>*** THE END *** </b>
</font> </center>